In [ ]:
def get_library_path()->str:

    cwd = os.getcwd()
    lst = cwd.split(os.sep)[:-1]

    path_lib = f'{os.sep}'

    for elem in lst:
        path_lib = os.path.join(path_lib, elem)

    return path_lib

# imports

import os
import sys

path_to_lib = get_library_path()

sys.path.append(path_to_lib)
import luxgiant_clinical.TwoCatAnalysis as two

import pandas as pd

In [ ]:
# load data

folder_path = os.path.join(path_to_lib, 'data/source')

df = pd.read_csv(
    os.path.join(folder_path, 'cleaned_file.csv'), 
    low_memory=False
)

In [ ]:
# filter data to keep only patients with disease duration information

mask_patients= (df['Status']=='Patient')
mask_length  = (~df['pdsl'].isnull()) 

df_cases = df[mask_patients & mask_length].reset_index(drop=True)
del df

df_cases["pdsl"] = df_cases["pdsl"].map({"<=5":'PD duration <=5 yrs', ">5":'PD duration >5 yrs'})

df_cases.shape

In [ ]:
df_on = df_cases[df_cases['hyonoff']=='On'].reset_index(drop=True)

df_on['Stage I-III']= df_on['hystage'].map({'Not severe':1, 'Severe':0})
df_on['Stage IV-V'] = df_on['hystage'].map({'Not severe':0, 'Severe':1})
df_on['UPDRS III-ON']= df_on['updrs_part_iii_total_score'].copy()

variables = ['participant_id', 'pdsl', 'Stage I-III', 'Stage IV-V', 'UPDRS III-ON']

stats_meas1 = {
    'n': ['Stage I-III', 'Stage IV-V'],
    'median' : ['UPDRS III-ON']
}
groups = ['PD duration <=5 yrs', 'PD duration >5 yrs']

# keep only variables for analysis

df_on = df_on[variables].copy()

In [ ]:
df_off = df_cases[df_cases['hyonoff']=='Off'].reset_index(drop=True)

df_off['Stage I-III']= df_off['hystage'].map({'Not severe':1, 'Severe':0})
df_off['Stage IV-V'] = df_off['hystage'].map({'Not severe':0, 'Severe':1})
df_off['UPDRS III-OFF']= df_off['updrs_part_iii_total_score'].copy()

variables = ['participant_id', 'pdsl', 'Stage I-III', 'Stage IV-V', 'UPDRS III-OFF']

stats_meas2 = {
    'n': ['Stage I-III', 'Stage IV-V'],
    'median' : ['UPDRS III-OFF']
}

# keep only variables for analysis

df_off = df_off[variables].copy()

In [ ]:
summary1 = [
    two.report_proportion(
        data_df    =df_off,
        variables  =stats_meas2['n'],
        groups     =groups,
        grouping_by='pdsl',
        subheader  ='H & Y OFF'
        ),
    two.report_proportion(
        data_df    =df_on,
        variables  =stats_meas1['n'],
        groups     =groups,
        grouping_by='pdsl',
        subheader  ='H & Y ON'
    ),
    two.report_median_iqr(
        data_df    =df_on,
        variables  =stats_meas1['median'],
        groups     =groups,
        grouping_by='pdsl'
    ),
    two.report_median_iqr(
        data_df    =df_off,
        variables  =stats_meas2['median'],
        groups     =groups,
        grouping_by='pdsl'
    )
]

In [ ]:
variables_dict = {
    "total_score_for_moca": "MOCA Score",
    "total_score_for_bdi" : "BDI Score",
}
variables_1 = ['participant_id', 'pdsl'] + list(variables_dict.keys())

# statistical measures
stats_meas3 = {
    'mean'  : ["MOCA Score"],
    'median': ["BDI Score"],
}

# keep only variables for analysis

df_cases_1 = df_cases[variables_1].copy()
df_cases_1 = df_cases_1.rename(columns=variables_dict)

In [ ]:
summary2 = [ 
    two.report_mean_std(
        data_df    =df_cases_1,
        variables  =stats_meas3['mean'],
        groups     =groups,
        grouping_by='pdsl'
    ),
    two.report_median_iqr(
        data_df    =df_cases_1,
        variables  =stats_meas3['median'],
        groups     =groups,
        grouping_by='pdsl'
    )
]

In [ ]:
df_10 = pd.concat(
    summary1 + summary2, axis=0, ignore_index=True
)
df_10 = two.final_formatter(df_10, groups)

df_10.to_csv(os.path.join(path_to_lib, 'data/auxiliar/aux_table_10.csv'), index=False)
df_10